In [3]:
from transformers import pipeline 
from bs4 import BeautifulSoup
import requests

In [37]:
def get_website_text(URL):
    # Get the raw text from a URL
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, 'html.parser')
    results = soup.find_all(['h2', 'p'])
    text  = [result.text for result in results]

    # Join all the texts together
    article = ' '.join(text)

    # Break up blog post into sentences
    article = article.replace('.', '.<eos>')
    article = article.replace('!', '!<eos>')
    article = article.replace('?', '?<eos>')
    sentences = article.split('<eos>')
    return sentences

def chunk_text(sentences):
    max_chunk = 500
    current_chunk = 0
    chunks = []

    # Create chunks with not more than 500 words
    for sentence in sentences:
        if len(chunks) == current_chunk + 1:
            if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
                chunks[current_chunk].extend(sentence.split(' '))
            else:
                current_chunk += 1
                chunks.append(sentence.split(' '))
        
        else:
            print(current_chunk)
            chunks.append(sentence.split(' '))

    # Join chunks back together
    for chunk_id in range(len(chunks)):
        chunks[chunk_id] = ' '.join(chunks[chunk_id])

    return chunks

def load_and_summarize(chunks, max_length, min_lenght):
    # Load in the summarizer
    summarizer = pipeline('summarization')

    # Write a summary for each chunk
    res = summarizer(chunks, max_length=max_length, min_length=min_lenght, do_sample=False)

    # Append the individual chunks back together
    summary = ' '.join(summ['summary_text'] for summ in res)
    return summary

In [ ]:
# Paste URL of a blog article here
URL = "url-goes-here"

# Summarizes the article
sentences = get_website_text(URL)
chunks = chunk_text(sentences)
summary = load_and_summarize(chunks, 60, 30)

In [53]:
# Save summary to a text file
with open('blog_summary.txt', 'w') as f:
    f.write(summary)